**disallowedSynonym.ipynb**

narrowSynonym or broadSynonym defined in Format subontology.

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/disallowedSynonym.md

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

In [77]:
import sys
from rdflib import ConjunctiveGraph, Namespace

# Constants for script return value as per https://github.com/edamontology/edamverify.
NOERR = 0
INFO  = 1
WARN  = 2
ERROR = 3

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")

Loading graph ...done!


Define SPARQL query to extract ID, term and subset of all concepts. Run the query.

In [78]:
# Compile SPARQL query
query_term = """
SELECT ?id ?term ?exact_syn ?narrow_syn ?broad_syn WHERE
{
?id rdfs:label ?term .
#OPTIONAL {?id oboInOwl:hasNarrowSynonym ?narrow_syn .}
#OPTIONAL {?id oboInOwl:hasBroadSynonym ?broad_syn .}
FILTER regex(str(?id), "format_")
}
"""




# Run SPARQL query and collate results
errfound = False    
report = list()
results = g.query(query_term)

Analyse results of query.

**NB.1:**
...



In [79]:
for r in results :
    
    id     = str(r['id'])
    term   = str(r['term'])
    narrow_syn = str(r['narrow_syn']) 
    broad_syn = str(r['broad_syn']) 
            
#    print(str(r['id']), "   ", str(r['term']), "   ",  "exact:", exact_syn)

    print(str(r['id']), "   ", str(r['term']), "   narrow:", narrow_syn, "   broad:", broad_syn)
    
     
    print()
    print("narrowSynonym:", narrow_syn)
    if(narrow_syn==None):
        print("narrow_syn == None")
    if(narrow_syn!=None):
        print("narrow_syn != None")
    if (narrow_syn is None):
        print("narrow_syn is None")
    if (narrow_syn is not None):
        print("narrow_syn is not None")
    if (narrow_syn == "None"):
        print('narrow_syn == "None"')
    if (narrow_syn != "None"):
        print('narrow_syn != "None"')
    
    print()

    if narrow_syn:
        errfound = True
        report.append("Disallowed synonym ::: " + id +  ' (' + term + ')' + " ::: " + "narrowSynonym:" + "   " + narrow_syn)

    if broad_syn:
        errfound = True
        report.append("Disallowed synonym ::: " + id +  ' (' + term + ')' + " ::: " + "broadSynonym:" + "   " + broad_syn)

http://edamontology.org/format_1912     Nexus format    narrow: None    broad: None

narrowSynonym: None
narrow_syn != None
narrow_syn is not None
narrow_syn == "None"

http://edamontology.org/format_1607     EcoCyc gene report format    narrow: None    broad: None

narrowSynonym: None
narrow_syn != None
narrow_syn is not None
narrow_syn == "None"

http://edamontology.org/format_3507     Document format    narrow: None    broad: None

narrowSynonym: None
narrow_syn != None
narrow_syn is not None
narrow_syn == "None"

http://edamontology.org/format_3910     trr    narrow: None    broad: None

narrowSynonym: None
narrow_syn != None
narrow_syn is not None
narrow_syn == "None"

http://edamontology.org/format_3166     Biological pathway or network report format    narrow: None    broad: None

narrowSynonym: None
narrow_syn != None
narrow_syn is not None
narrow_syn == "None"

http://edamontology.org/format_3688     SBtab    narrow: None    broad: None

narrowSynonym: None
narrow_syn != None


Write report and return approriate value.

In [80]:
# Return exit code (raises exception)
if errfound:
    print("Synonym (disallowed) defined for these Format concepts:")
    print("\n".join(report))
    sys.exit(ERROR)
else:
    print("No issues found.")
    sys.exit(NOERR)


Synonym (disallowed) defined for these Format concepts:
Disallowed synonym ::: http://edamontology.org/format_1912 (Nexus format) ::: narrowSynonym:   None
Disallowed synonym ::: http://edamontology.org/format_1912 (Nexus format) ::: broadSynonym:   None
Disallowed synonym ::: http://edamontology.org/format_1607 (EcoCyc gene report format) ::: narrowSynonym:   None
Disallowed synonym ::: http://edamontology.org/format_1607 (EcoCyc gene report format) ::: broadSynonym:   None
Disallowed synonym ::: http://edamontology.org/format_3507 (Document format) ::: narrowSynonym:   None
Disallowed synonym ::: http://edamontology.org/format_3507 (Document format) ::: broadSynonym:   None
Disallowed synonym ::: http://edamontology.org/format_3910 (trr) ::: narrowSynonym:   None
Disallowed synonym ::: http://edamontology.org/format_3910 (trr) ::: broadSynonym:   None
Disallowed synonym ::: http://edamontology.org/format_3166 (Biological pathway or network report format) ::: narrowSynonym:   None
Dis

SystemExit: 3

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
